This code generates the model SNe for each model type and prints the fraction of emission in HII regions for the controls in the paper.


In [1]:
#import packages and initialize settings
import time
import os
import numpy as np
from astropy.io import fits, ascii
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join
from reproject import reproject_interp
import aplpy
import scipy
from scipy.interpolate import interp1d

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

color1 = '#540D6E'
color2 = '#EE4266'
color3 = '#FFD23F'

import sys
sys.path.append('./Python')
from nearest import findNearest
from deprojectGalaxy import deproject
from SNeGenerationModels import runModels
from getMapValue import getValue

In [2]:
imgName = "SFRcopt"
ext = "HA_FLUX_CORR"


#Get Galaxy names, distances, and fits file locations

# Unpack master table to get SNe information
GalaxyData = Table.read('../Data/0.MUSEData.csv', format='csv')
SampleData = Table.read('../Data/2.MUSESampleCat.csv', format='csv') 
SampleData.remove_column("Distance")
SampleData.remove_column("MUSENatMap")
SampleData.remove_column("MUSE150pcMap")
SampleData.remove_column("MUSEHII")
SampleData.remove_column("MUSERes")
SampleData.remove_column("IRACfiles")
SampleData.remove_column("Incl")
SampleData.remove_column("coptMap")

database = join(GalaxyData, SampleData, keys = "Galaxy", join_type = 'inner')
centerCoords = [(database["GalRa"][i], database["GalDec"][i]) for i in range(len(database))]   

database[0:8]


Galaxy,Distance,GalRa,GalDec,PosAng,Incl,MUSENatRes,MUSEPhysRes,MUSEmapNat,MUSEmap150pc,MUSE_HII_reg,IRAC_files,SFRcopt,SFR15asec,maps150,maps1kpc,CO21File,eCO21File,Supernova,Type,Ra,Dec,Intensity,CO21,eCO21,InSample
str7,float64,float64,float64,float64,float64,float64,int64,str71,str70,str50,str40,str58,str60,str64,str63,str72,str73,str11,str12,float64,float64,float64,str72,str73,str4
NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,84,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1087-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,SN1995V,II,41.61152777777777,-0.4987861111111111,8.839672735197591e+39,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,True
NGC1300,18.99,49.920815,-19.411114,278.0,31.8,0.89,83,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1300-0.89asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1300-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1300_nebulae_mask_V2.fits,../Data/IRAC/ngc1300_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1300_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1300_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1300_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1300_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc1300_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1300_12m+7m+tp_co21_150pc_broad_emom0.fits,SN2022acko,II,49.91245833333333,-19.39518888888889,3.445745101042276e+38,../../GalaxyFiles/ALMACO2-1/ngc1300_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1300_12m+7m+tp_co21_150pc_broad_emom0.fits,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,76,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,../Data/IRAC/ngc1365_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1365_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_emom0.fits,SN2012fr,Ia,53.40057916666666,-36.12676944444445,8.904660571183651e+37,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_emom0.fits,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,76,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,../Data/IRAC/ngc1365_irac1_atGauss4.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1365_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1365_sfr_1kpc.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_emom0.fits,SN2001du,II,53.3713125,-36.142111111111106,1.3530435354080875e+39,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1365_12m+7m+tp_co21_150pc_broad_e

In [3]:
# Functions
def angDistToPc(x,galDist):
    return(galDist*10**6*np.tan(x*np.pi/180))

def nearestHII(galaxy, HII_file, centerCoord, pa, incl, galDist, SNra, SNdec):
    HII_val = []
    if os.path.isfile(HII_file):

        # read in fits files
        hdu_HII = pyfits.open(HII_file)
        HIIMap  = hdu_HII[0].data

        #Convert x & y pixels to ra and dec
        wcs      = WCS(hdu_HII[0].header, naxis=2)
        naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
        grid     = np.indices((naxis[1],naxis[0]))
        ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)

        #deproject ra and dec to dx and dy
        radius, projang, dx, dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=ra, dec=dec, return_offset=True)

        # Get the value from the HII map at the SN location
        pix_x, pix_y = wcs.wcs_world2pix(SNra, SNdec, 0, ra_dec_order=True)
        pix_x = [int(x) for x in pix_x]
        pix_y = [int(x) for x in pix_y]
        
        for i in range(len(SNra)):
            HII_val.append(HIIMap[pix_y[i], pix_x[i]])

        
        #flatten data structures 
        f_HII  = HIIMap.flatten()
        f_ra   = ra.flatten()
        f_dec  = dec.flatten()    
        f_dx   = dx.flatten()
        f_dy   = dy.flatten()

        #remove nans
#         keep  = np.where(np.isfinite(f_HII))
        keep  = np.where(f_HII >= 0)
        map_ra    = f_ra[keep]
        map_dec   = f_dec[keep]
        map_HII   = f_HII[keep]
        map_dx    = f_dx[keep]
        map_dy    = f_dy[keep]

        SN_Rad, SN_PA, SN_dx, SN_dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=SNra, dec=SNdec, return_offset=True) 
        
        nearestHII_ang, idx = findNearest(map_dx, SN_dx, map_dy, SN_dy)
        HIIra, HIIdec = map_ra[idx], map_dec[idx]
        nearestHII_pc = angDistToPc(nearestHII_ang,galDist)
        
        #Check if the SN is in an HII region and adjust nearest HII region value accordingly
        for i in range(len(SNra)):
            if HII_val[i] > 0:
                nearestHII_pc[i] = 0.
        
#         print("Nearest HII region [pc]", nearestHII_pc, galaxy)

    else:
        print("No file for ", galaxy)

        nearestHII_pc = float("nan")
        HIIra = float("nan")
        HIIdec = float("nan")
        
    return(nearestHII_pc, HIIra, HIIdec)


In [5]:
#group sample by galaxy
gal_groups = database.group_by('Galaxy')
galaxies = gal_groups.groups.keys
indices = gal_groups.groups.indices
indices = indices.tolist()
indices.append(0)

gs, imgs, HIIimgs,IRACimgs,COimgs,eCOimgs, names, types, ras, decs, dists, res = [],[],[],[],[],[],[],[],[],[],[],[]
galRa,galDec,galPA,galIncl = [],[],[],[]

for i in range(len(indices)-2):
    SNnames, SNtypes, SNras, SNdecs = [],[],[],[]
    gs.append(gal_groups["Galaxy"][indices[i]])
    imgs.append(gal_groups[imgName][indices[i]])
    HIIimgs.append(gal_groups["MUSE_HII_reg"][indices[i]])
    IRACimgs.append(gal_groups["IRAC_files"][indices[i]])
    COimgs.append(gal_groups["CO21File"][indices[i]])
    eCOimgs.append(gal_groups["eCO21File"][indices[i]])
    dists.append(gal_groups["Distance"][indices[i]])
    res.append(gal_groups["MUSENatRes"][indices[i]])
    galRa.append(gal_groups["GalRa"][indices[i]])
    galDec.append(gal_groups["GalDec"][indices[i]])
    galPA.append(gal_groups["PosAng"][indices[i]])
    galIncl.append(gal_groups["Incl"][indices[i]])
    
    #if gal_groups["SN_name"][indices[i]] != "SN1989B" and gal_groups["SN_name"][indices[i]] != "SN1973R":
    # Attach first SN per galaxy to a list
    SNnames.append(gal_groups["Supernova"][indices[i]])
    SNtypes.append(gal_groups["Type"][indices[i]])
    SNras.append(gal_groups["Ra"][indices[i]])
    SNdecs.append(gal_groups["Dec"][indices[i]])
    j = 1
    # attach additional  SN to list when galaxy has more than one
    while(indices[i] + j < indices[i+1]):    
        #if gal_groups["SN_name"][indices[i]+j] != "SN1989B" and gal_groups["SN_name"][indices[i]+j] != "SN1973R":
        SNnames.append(gal_groups["Supernova"][indices[i]+j])
        SNtypes.append(gal_groups["Type"][indices[i]+j])
        SNras.append(gal_groups["Ra"][indices[i]+j])
        SNdecs.append(gal_groups["Dec"][indices[i]+j])
        j += 1
    names.append(SNnames)
    types.append(SNtypes)
    ras.append(SNras)
    decs.append(SNdecs)

print(gs)
print(ras)

['NGC1087', 'NGC1300', 'NGC1365', 'NGC1433', 'NGC1566', 'NGC1672', 'NGC3627', 'NGC4254', 'NGC4303', 'NGC4321']
[[41.61152777777777], [49.91245833333333], [53.40057916666666, 53.3713125, 53.381876388888884, 53.38349999999999], [55.52640625], [64.9725, 64.99512499999999, 65.0058875], [71.423975, 71.45607916666667], [170.04809375, 170.05796666666666, 170.07957933333336, 170.07072916666667, 170.0592513888889], [184.71836111111114, 184.71066666666667, 184.71690625, 184.70291666666668], [185.48760972222217, 185.47536458333335, 185.50184583333336, 185.4698125, 185.46016458333332, 185.48041875, 185.48991666666666], [185.72488194444443, 185.74543194444445, 185.6970833333333, 185.72892708333336, 185.73393055555553]]


In [121]:
def divide(x,y): 
    return(float(x)/float(y))

# # run models 1-4 and store results
galCol, counter, int_mod1, int_mod2, int_mod3, near_mod1, near_mod2, near_mod3 = [],[],[],[],[],[],[],[]
HaTot, HaHII, SDTot, SDHII, COHII, COTot = [],[],[],[],[],[]
HaPix, HaHIIPix, HaHIICOPix, HaCOPix = [],[],[],[]
SDPix, SDHIIPix, SDHIICOPix, SDCOPix = [],[],[],[]
COPix, COHIIPix = [],[]
HaHIICO, HaCO, SDCO, SDHIICO = [],[],[],[]

for i in range(len(gs)):
    
    # Using MUSE map for coverage and to measure H-alpha emission
    img_hdu = pyfits.open(imgs[i])
    img_map = img_hdu[ext].data
    
    #Get IRAC maps to look at the stellar populations emission
    IRAC_hdu = pyfits.open(IRACimgs[i])
    IRAC_map = IRAC_hdu[0].data

    #Get CO maps to look at the CO21 emission
    CO_hdu = pyfits.open(COimgs[i])
    CO_map = CO_hdu[0].data
    
    #Get eCO maps to look at the CO21 emission error
    eCO_hdu = pyfits.open(eCOimgs[i])
    eCO_map = eCO_hdu[0].data  
    # clip so that we have no negatives or zeroes
    eCO_clip_map = np.nan_to_num(eCO_map, copy=True, nan=0.0, posinf=None, neginf=None)
    eCO_clip_map = np.clip(eCO_clip_map, 0.1, np.max(eCO_clip_map))
    
    # Make CO S/N maps
    CO_StN_map = CO_map/eCO_clip_map
#         CO_StN_map = map(divide, CO_map, eCO_clip_map)

    CO_StN_hdu = pyfits.PrimaryHDU(data=CO_StN_map, header=CO_hdu[0].header)
    
    #Get HII regions masks
    hdu_HII = pyfits.open(HIIimgs[i])
    HIIMap  = hdu_HII[0].data    
    
    # Reproject IRAC maps
    IRAC_repro, IRAC_foot = reproject_interp(IRAC_hdu, img_hdu[ext].header)

    # Reproject CO maps
    CO_repro, CO_foot = reproject_interp(CO_hdu, img_hdu[ext].header)
    
    # Reproject CO StN maps
    CO_StN_repro, CO_foot = reproject_interp(CO_StN_hdu, img_hdu[ext].header)

    #Convert x & y pixels to ra and dec
    wcs      = WCS(hdu_HII[0].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)
            
    #flatten data structures 
    f_HII    = HIIMap.flatten()
    f_ra     = ra.flatten()
    f_dec    = dec.flatten()    
    f_Ha     = img_map.flatten()
    f_SD     = IRAC_repro.flatten() 
    f_CO     = CO_repro.flatten()
    f_CO_StN = CO_StN_repro.flatten()
    
    #remove nans for Halpha only
    keep     = np.where(np.isfinite(f_Ha))
    map_HII  = f_HII[keep]
    map_Ha   = f_Ha[keep] 
    
    #remove nans for Halpha and IRAC overlap
    keep_SD    = np.where(np.isfinite(f_Ha) & np.isfinite(f_SD))
    map_Ha_SD  = f_Ha[keep_SD] 
    map_SD     = f_SD[keep_SD]
    map_HII_SD = f_HII[keep_SD]

    #remove nans for Halpha and CO overlap
    keep_CO     = np.where(np.isfinite(f_Ha) & np.isfinite(f_CO))
    map_Ha_CO   = f_Ha[keep_CO] 
    map_CO      = f_CO[keep_CO]
    map_HII_CO  = f_HII[keep_CO]
    
    #remove nans for Halpha and CO_StN calculation overlap
    keep_CO_StN    = np.where(np.isfinite(f_Ha) & np.isfinite(f_CO_StN) & np.isfinite(f_HII))
    map_Ha_CO_StN  = f_Ha[keep_CO_StN] 
    map_CO_StN     = f_CO[keep_CO_StN] 
    map_HII_CO_StN = f_HII[keep_CO_StN]

    # sum up the total emission
    HaPix.append(len(map_Ha))
    SDPix.append(len(map_SD))
    COPix.append(len(map_CO))
    HaTot.append(np.sum(map_Ha_CO_StN)) # check this
    SDTot.append(np.sum(map_SD))
    COTot.append(np.sum(map_CO))
    
    # Isolate HII regions from full coverage Ha map 
    keep_HII    = np.where(map_HII >= 0)
    keep_no_HII = np.where(map_HII < 0) 
        
    #Isolate HII regions and real values from SD and CO maps
    keep_SD_HII     = np.where(map_HII_SD >= 0 & np.isfinite(map_SD))
    keep_SD_no_HII  = np.where(map_HII_SD < 0 & np.isfinite(map_SD))
    keep_CO_HII     = np.where(map_HII_CO >= 0 & np.isfinite(map_CO))
    keep_CO_no_HII  = np.where(map_HII_CO < 0 & np.isfinite(map_CO))

    # Isolate HII regions from overlap coverage of Ha map and CO map
    keep_HII_CO_StN    = np.where(map_HII_CO_StN >= 0)
    keep_no_HII_CO_StN = np.where(map_HII_CO_StN < 0)     

    # Emission in HII regions
    map_SD_HII     = map_SD[keep_SD_HII]
    map_SD_no_HII  = map_SD[keep_SD_no_HII]
    map_CO_HII     = map_CO[keep_CO_HII]
    
    # Halpha, CO, and HII 
    map_Ha_HII        = map_Ha_CO_StN[keep_HII_CO_StN]
    map_CO_StN_resize = map_CO_StN[keep_HII_CO_StN] #resizing this
    map_Ha_no_HII     = map_Ha_CO_StN[keep_no_HII_CO_StN]

    # Isolate StN > 3 values for coincident CO *WITH* HII
    keep_CO_StN  = np.where(map_CO_StN_resize >= 3.)
    map_Ha_HII_CO = map_Ha_HII[keep_CO_StN] 
    map_SD_HII_CO = map_SD_HII[keep_CO_StN]
    # Isolate StN > 3 values for coincident CO *WITHOUT* HII
    map_Ha_no_HII_CO = map_Ha_no_HII[keep_CO_StN]
    map_SD_no_HII_CO = map_SD_no_HII[keep_CO_StN]
    
    # sum up the HII region emission
    HaHIIPix.append(len(map_Ha_HII))
    SDHIIPix.append(len(map_SD_HII))
    COHIIPix.append(len(map_CO_HII))
    HaHII.append(np.sum(map_Ha_HII))
    SDHII.append(np.sum(map_SD_HII))
    COHII.append(np.sum(map_CO_HII)) 
    HaHIICO.append(np.sum(map_Ha_HII_CO))
    HaCO.append(np.sum(map_Ha_no_HII_CO))
    SDHIICO.append(np.sum(map_SD_HII_CO))
    SDCO.append(np.sum(map_SD_no_HII_CO))
    HaHIICOPix.append(len(map_Ha_HII_CO))
    SDHIICOPix.append(len(map_SD_HII_CO))
    HaCOPix.append(len(map_Ha_no_HII_CO))
    SDCOPix.append(len(map_SD_no_HII_CO))
    
    # get maps with real values for models
    keep = np.where(np.isfinite(f_Ha) & np.isfinite(f_SD))
    IRAC      = f_SD[keep]
    star_ra   = f_ra[keep]
    star_dec  = f_dec[keep]
    
    
    raArr1, decArr1, dxArr1, dyArr1, intArr1 = runModels(gs[i], imgs[i], ext, centerCoords[i], galPA[i], galIncl[i], dists[i], modelType=1)
    raArr2, decArr2, dxArr2, dyArr2, intArr2 = runModels(gs[i], imgs[i], ext, centerCoords[i], galPA[i], galIncl[i], dists[i], modelType=2)
    raArr3, decArr3, dxArr3, dyArr3, intArr3 = runModels(gs[i], imgs[i], ext, centerCoords[i], galPA[i], galIncl[i], dists[i], modelType=3, starLight = IRAC, starRa = star_ra, starDec = star_dec)

    n_mod1_dist, n_mod1_ra, n_mod1_dec = nearestHII(gs[i], HIIimgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], raArr1, decArr1)
    n_mod2_dist, n_mod2_ra, n_mod2_dec = nearestHII(gs[i], HIIimgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], raArr2, decArr2)
    n_mod3_dist, n_mod3_ra, n_mod3_dec = nearestHII(gs[i], HIIimgs[i], centerCoords[i], galPA[i], galIncl[i], dists[i], raArr3, decArr3)

    
    if (type(n_mod1_dist) != float and type(n_mod1_dist) != np.float64):
        for j in range(len(n_mod1_dist)):
            galCol.append(gs[i])
            counter.append(j)
            near_mod1.append(n_mod1_dist[j])
            near_mod2.append(n_mod2_dist[j])
            near_mod3.append(n_mod3_dist[j])
    else:
        for j in range(1000):
            galCol.append(gals[i])
            counter.append(j)
            near_mod1.append(np.float('nan'))
            near_mod2.append(np.float('nan'))
            near_mod3.append(np.float('nan'))
            
    intArr1    = np.nan_to_num(np.clip(intArr1,0,np.max(intArr1)))
    intArr2    = np.nan_to_num(np.clip(intArr2,0,np.max(intArr2)))
    intArr3    = np.nan_to_num(np.clip(intArr3,0,np.max(intArr3)))
    
    int_mod1.append(intArr1)
    int_mod2.append(intArr2)
    int_mod3.append(intArr3)
    
int_mod1 = np.hstack(int_mod1)
int_mod2 = np.hstack(int_mod2)
int_mod3 = np.hstack(int_mod3)
    
results = Table({"Galaxy": galCol,
                "ModelNum": counter,
                "near_Mod1_dist": near_mod1,
                "near_Mod2_dist": near_mod2,
                "near_Mod3_dist": near_mod3,
                "intMod1":int_mod1,
                "intMod2":int_mod2,
                "intMod3":int_mod3})

results.write("../Data/AllModels.csv", format = "csv", overwrite = True)

        
        
    

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
r_int: [1.15858782e+

r_int: [6.21806133e+39 6.78871547e+40 1.23004366e+41 4.24376373e+39
 1.19534253e+40 1.77941246e+40 5.43596974e+39 2.61340548e+40
 1.03706381e+40 2.30367331e+39 4.38203115e+40 4.92358182e+40
 3.92125328e+39 3.51753897e+40 1.32233494e+41 4.69450109e+40
 6.17287005e+38 4.39787183e+39 1.22488359e+40 3.24123495e+40
 3.79577624e+39 7.93405340e+39 7.86504229e+39 1.42163441e+41
 1.93249332e+40 6.87085374e+39 2.57079502e+40 6.41826803e+40
 1.44173798e+41 3.40168341e+37 1.19268395e+41 5.87164994e+39
 8.31320282e+40 3.42781280e+37 8.16966326e+39 1.07016208e+40
 6.19526792e+40 3.92803787e+39 1.36141238e+39 1.14057326e+40
 2.63239914e+40 2.61446619e+40 8.55712618e+39 4.10741935e+40
 1.60145054e+39 6.85834699e+40 1.39637716e+41 2.20718961e+39
 2.23866814e+40 4.68826618e+39 8.02937160e+39 7.03411700e+40
 1.81770364e+40 2.29145849e+40 3.26078212e+39 1.79021201e+39
 2.17144585e+40 2.40111244e+40 1.09312005e+40 1.82003410e+40
 2.62682595e+40 8.34561511e+39 3.78414437e+41 4.77016867e+39
 3.88430914e+39 2

r_int: [5.14735391e+40 7.00681458e+38 2.06550469e+39 3.54160137e+37
 2.30271438e+38 1.09821690e+40 6.78871203e+38 6.43974917e+38
 4.47119810e+39 3.28218766e+37 1.57981247e+40 3.96054726e+37
 7.77422128e+39 1.58440583e+39 1.48528730e+40 6.51996083e+39
 1.13799864e+37 5.49405395e+38 2.96164074e+39 2.06145947e+40
 3.87822250e+39 5.09338928e+38 4.57075292e+39 3.36469501e+38
 8.52809245e+38 7.17830680e+38 4.25043605e+39 8.31726490e+39
 3.53934254e+38 2.83704166e+38 3.36776412e+39 1.12288672e+38
 6.99803299e+37 3.74725336e+37 4.73276094e+38 2.04231865e+39
 2.01242014e+38 1.25885295e+39 2.52511509e+39 6.58022305e+39
 3.49518551e+39 1.26021337e+39 1.03800652e+39 3.85593486e+38
 4.81841285e+39 9.52993967e+38 3.97265618e+39 1.26413860e+39
 1.52940945e+40 6.25677647e+39 1.28681333e+39 1.21446653e+38
 2.97617784e+39 6.41948034e+39 2.45911045e+37 6.71217483e+39
 7.26173516e+37 1.33939605e+39 2.79563832e+39 5.09974182e+38
 3.93966650e+39 3.27496631e+38 5.47301668e+38 3.86376992e+38
 9.07042990e+39 7

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
r_int: [1.41512464e+

r_int: [1.02958842e+40 2.18594231e+40 7.24421608e+40 1.41786928e+40
 5.30906959e+39 1.92417572e+39 5.71427352e+39 6.72614481e+39
 5.66862298e+39 3.77206307e+39 8.95067272e+39 8.24293864e+40
 8.70316550e+38 1.05958812e+40 5.65721310e+39 4.48377026e+39
 1.58962348e+39 4.37830095e+39 2.05837689e+39 3.17540264e+40
 2.23974515e+39 1.02404029e+39 2.63560703e+38 4.89720486e+38
 4.91967215e+39 1.03283773e+39 1.41497394e+39 1.19845581e+40
 3.49688235e+37 3.49748913e+40 2.27434724e+39 7.08818597e+39
 1.69158705e+40 1.28251216e+39 1.16846128e+40 2.99050127e+38
 4.37190987e+40 8.67938192e+38 7.59275094e+38 1.05953775e+39
 2.70409806e+39 3.51460011e+37 1.07125765e+40 2.35711945e+40
 1.75730784e+37 2.64720950e+39 5.62082495e+38 9.01331286e+38
 7.13289221e+40 6.19113751e+40 3.51583492e+40 2.78788543e+39
 3.97054894e+40 5.29941206e+39 4.45117747e+40 1.93819929e+40
 1.51677254e+38 2.38760843e+39 1.04912595e+38 1.04063316e+38
 8.66421544e+39 1.29734723e+39 1.40544853e+40 1.65406904e+40
 5.48280259e+38 8

...

r_int: [2.10649268e+37 6.21109680e+37 2.57967821e+37 8.87485551e+37
 6.48461662e+39 2.53958488e+37 4.51650513e+37 2.57713398e+37
 5.36913460e+37 3.78983674e+39 5.67411784e+37 5.86694600e+37
 2.56757203e+37 1.21347678e+38 9.48423858e+37 8.27788782e+37
 7.92794179e+37 2.05487823e+38 3.36818404e+37 3.56140573e+38
 2.20467963e+39 4.72750663e+37 3.33906120e+37 1.38928578e+38
 8.74974790e+37 5.05560863e+37 2.06739192e+37 1.90085927e+37
 1.36191405e+38 5.81419880e+37 3.60605096e+37 5.40978404e+39
 2.45525621e+38 4.20840849e+37 5.82194846e+36 7.64328722e+37
 3.02622862e+39 6.30097791e+37 4.30933967e+37 1.13581363e+38
 2.77143435e+37 1.05304861e+38 4.71157974e+37 4.45335089e+37
 3.15455281e+37 1.66338084e+39 9.70341565e+37 4.43737588e+37
 1.05405727e+38 7.28974878e+38 7.95961317e+37 4.00829735e+37
 5.63967673e+37 6.64680251e+37 2.65038001e+38 1.82598722e+37
 1.50113810e+38 7.09014768e+37 1.69320014e+37 3.90313397e+37
 9.31335174e+37 1.93706142e+37 7.90347785e+37 6.91919782e+37
 4.36398544e+39 2

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
r_int: [1.58621940e+

r_int: [1.18723846e+41 1.94565914e+39 8.89880890e+37 4.01275947e+40
 8.32662612e+40 2.03457685e+40 5.32457517e+39 3.87022741e+40
 6.94739609e+39 8.90067224e+39 1.62838045e+40 1.15236895e+40
 1.78211518e+40 4.96112441e+40 1.05679352e+40 1.36376329e+40
 8.44216106e+38 5.22696525e+38 5.52748005e+39 1.07507093e+39
 1.86835656e+40 2.98634211e+39 3.50915082e+39 7.37066216e+40
 1.08153636e+40 3.47269768e+39 8.23836900e+39 5.99308483e+40
 4.42290477e+40 4.10521443e+40 4.46113417e+39 2.94157166e+39
 6.26387763e+39 2.27353507e+40 1.26161420e+39 3.69735356e+40
 1.19691173e+41 4.53544669e+40 3.04971883e+40 3.57436136e+40
 5.61916476e+38 5.61839866e+39 1.11530030e+40 7.49148496e+40
 2.14672428e+41 1.29428973e+40 4.46512774e+40 7.79945370e+40
 1.18293031e+41 2.02142490e+40 6.95005928e+38 1.70782141e+41
 7.57171274e+40 3.89740757e+40 7.48530816e+39 1.34663389e+40
 1.62830552e+40 6.75772473e+39 2.52358748e+40 3.46206156e+40
 1.89283478e+39 1.74610525e+40 5.13377334e+38 6.29978031e+39
 4.13076133e+40 1

r_int: [6.81015440e+38 1.33251894e+39 2.73201582e+38 6.30308962e+38
 2.45911196e+38 7.65371524e+38 8.08362831e+38 1.37639711e+39
 1.00740698e+39 3.29676398e+39 6.95847491e+38 2.51392502e+38
 1.57126773e+39 3.10617701e+38 2.05769413e+38 9.05160600e+38
 9.40650160e+39 6.90642884e+39 1.01651516e+40 2.12901157e+40
 5.20892152e+39 3.98476606e+38 3.21009382e+38 8.43855592e+38
 2.28731674e+39 9.65551690e+38 5.26946358e+39 7.96985438e+37
 4.35405275e+38 2.42034754e+38 6.49673023e+37 1.40080221e+38
 8.45983864e+38 1.43377355e+38 5.59593609e+38 2.75361192e+38
 7.00761357e+38 5.27738502e+38 1.78444066e+39 7.62012893e+38
 8.12700487e+38 6.89114708e+38 6.52889600e+38 1.06924790e+39
 4.02945940e+38 2.00449731e+38 7.94697596e+38 3.95206061e+38
 2.86161087e+39 1.50894466e+39 1.33964387e+39 3.31780040e+39
 1.81853130e+38 9.45030982e+38 1.61779698e+38 1.57789938e+38
 1.10153534e+39 1.54638062e+39 7.32348456e+38 1.37648844e+38
 7.06376965e+37 1.75933928e+39 9.49615105e+38 2.21520953e+38
 2.83866683e+39 5

FileNotFoundError: [Errno 2] No such file or directory: '../../GalaxyFiles/ALMACO2-1/ngc1672_12m+7m+tp_co21_150pc_broad_mom0.fits'

In [123]:
print("EMISSION")
print("--------")
print("Fraction of Ha Emission in HII regions",np.sum(HaHII)/np.sum(HaTot))
print()
print("Fraction of SD Emission in HII regions",np.sum(SDHII)/np.sum(SDTot))
print()
print("Fraction of CO Emission in HII regions",np.sum(COHII)/np.sum(COTot))
print("--------")
print()

print("EMISSION WITH/OUT CO HII")
print("------------------------")

print("Fraction of Ha emission in HII regions with coincident CO21 emission",
      np.sum(HaHIICO)/np.sum(HaTot))
print("Fraction of Ha emission NOT in HII regions with coincident CO21 emission",
      np.sum(HaCO)/np.sum(HaTot))
print()
print("Fraction of IRAC emission in HII regions with coincident CO21 emission",
      np.sum(SDHIICO)/np.sum(SDTot))
print("Fraction of IRAC emission NOT in HII regions with coincident CO21 emission",
      np.sum(SDCO)/np.sum(SDTot))
print()
print("--------")

print("RANDOM MAP PIXELS")
print("-----------------")
print("We want the probability that a random pixel will be in HII, HII + CO, CO only")
print("Fraction of Pixels in HII regions",np.sum(HaHIIPix)/np.sum(HaPix))
print("Fraction of Pixels in HII regions with CO",np.sum(HaHIICOPix)/np.sum(HaPix))
print("Fraction of Pixels NOT in HII regions with CO",np.sum(HaCOPix)/np.sum(HaPix))

# print("Fraction of Ha Pixels in HII regions",np.sum(HaHIIPix)/np.sum(HaPix))
# print("Fraction of SD Pixels in HII regions",np.sum(SDHIIPix)/np.sum(SDPix))
# print("Fraction of CO Pixels in HII regions",np.sum(COHIIPix)/np.sum(COPix))
print()
print()
print("RANDOM LOCAL")
print("------------")
print("We want the probability that a random pixel will be in HII, HII + CO, CO only for 500 and 1000pc")

print()

EMISSION
--------
Fraction of Ha Emission in HII regions 0.7767012902020108

Fraction of SD Emission in HII regions 0.2740936778828819

Fraction of CO Emission in HII regions 0.5214378395455066
--------

EMISSION WITH/OUT CO HII
------------------------
Fraction of Ha emission in HII regions with coincident CO21 emission 0.7057954753510003
Fraction of Ha emission NOT in HII regions with coincident CO21 emission 0.005227285459855251

Fraction of IRAC emission in HII regions with coincident CO21 emission 0.1389295963044988
Fraction of IRAC emission NOT in HII regions with coincident CO21 emission 0.016266993930157677

--------
RANDOM MAP PIXELS
-----------------
We want the probability that a random pixel will be in HII, HII + CO, CO only
Fraction of Pixels in HII regions 0.0829867694638277
Fraction of Pixels in HII regions with CO 0.03496524253562819
Fraction of Pixels NOT in HII regions with CO 0.03496524253562819


RANDOM LOCAL
------------
We want the probability that a random pixel 

In [128]:
HaPix

[525699, 1039792, 1125348, 1295093, 696947]

In [42]:
import math
# get the real SN sample's distances (copied over from 1c)
realSNe = [0,130,524,88,0,201,0,120,100,0,0,5,0,0,29,0,0,11,0,7,0,0,44,0,36,22,14,214,63,8,13,0,0]

# Here make CDFs of the distances from the generated SNe to the nearest HII regions
modelResults = Table.read("../Data/AllModels.csv", format = 'csv')

intScale = 10.0**37

#distance CDFs
nm1 = [x for x in modelResults["near_Mod1_dist"] if math.isnan(x) == False]
nm2 = [x for x in modelResults["near_Mod2_dist"] if math.isnan(x) == False]
nm3 = [x for x in modelResults["near_Mod3_dist"] if math.isnan(x) == False]
#intensity CDFs
im1 = [x/intScale for x in modelResults["intMod1"] if math.isnan(x) == False]
im2 = [x/intScale for x in modelResults["intMod2"] if math.isnan(x) == False]
im3 = [x/intScale for x in modelResults["intMod3"] if math.isnan(x) == False]


In [ ]:
#original idea for how this should work
def randomPull(modelArr, numSNe=33, numRuns=1000):
    # pull intensity vals at locations of 33 model SNe 1000 times.    
    allPulls_cdf, allPulls_x = [],[]
    # iterate over 1000 runs
    for i in range(numRuns):
        singlePulls = []
        # iterate over number of SNe
        for j in range(numSNe):
            # choose a random distance from model SNe
            val = np.random.choice(modelArr)
            singlePulls.append(val)
        # sort the model SNe distances and interpolate then save
        sortedPulls_y = np.sort(singlePulls)
        sortedPulls_x = np.linspace(0,len(sortedPulls_y),len(sortedPulls_y))/len(sortedPulls_y)
        x = np.linspace(0,len(sortedPulls_y),1000)/len(sortedPulls_y)
        y = np.interp(x, sortedPulls_x, sortedPulls_y)
        allPulls_x.append(y)
        allPulls_cdf.append(x)

    # Find the 1000 closest random pull values to each dist value in model SN array       
    allPer16Vals, allPer50Vals, allPer84Vals, cdfVals = [],[],[],[]
    for i in range(numSNe):
        SNlocVals = []
        targCDF = (i + 1.)/numSNe
        for j in range(numRuns):
#             find the actual distance value that is the closest to the generated model values 
            sim_CDF, sim_x = allPulls_cdf[j], allPulls_x[j]
            sim_at_targ = np.interp(targCDF,sim_CDF,sim_x)
            SNlocVals.append(sim_at_targ)    

    # now find 16-84th percentiles from each of these local distributions around x value    
        cdfVals.append(targCDF)
        locVals = np.sort(SNlocVals)
        per16Vals = np.percentile(locVals,16)
        allPer16Vals.append(per16Vals)
        per50Vals = np.percentile(locVals,50)     
        allPer50Vals.append(per50Vals)
        per84Vals = np.percentile(locVals,84)
        allPer84Vals.append(per84Vals)

    return(allPer16Vals, allPer50Vals, allPer84Vals, cdfVals)

In [ ]:
#dist
nSMm1_16, nSMm1_50, nSMm1_84, nSMm1_CDF = randomPull(nm1)
nSMm2_16, nSMm2_50, nSMm2_84, nSMm2_CDF = randomPull(nm2)
nSMm3_16, nSMm3_50, nSMm3_84, nSMm3_CDF = randomPull(nm3)
#int
iSMm1_16, iSMm1_50, iSMm1_84, iSMm1_CDF  = randomPull(im1)
iSMm2_16, iSMm2_50, iSMm2_84, iSMm2_CDF  = randomPull(im2)
iSMm3_16, iSMm3_50, iSMm3_84, iSMm3_CDF  = randomPull(im3)


In [ ]:
realSNeInt = database["Intensity"]/intScale

import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette("rocket", n_colors=3, desat=None, color_codes=False)

fig, ax = plt.subplots(1,1, figsize=(6,4))

sns.ecdfplot(realSNe, color = "dimgrey",  label="Real SNe Sample", ax=ax,lw=2)
sns.ecdfplot(nm2, label="H-alpha", ls="--",lw=2,ax=ax)
sns.ecdfplot(nm3, label="Stars (3.4 Micron)", ls="-.",lw=2,ax=ax)
sns.ecdfplot(nm1, label="Random", ls=":",ax=ax,lw=2)
ax.fill_betweenx(y=nSMm2_CDF, x1=nSMm2_16,x2=nSMm2_84, alpha=0.2)
ax.fill_betweenx(y=nSMm3_CDF, x1=nSMm3_16,x2=nSMm3_84, alpha=0.2)
ax.fill_betweenx(y=nSMm1_CDF, x1=nSMm1_16,x2=nSMm1_84, alpha=0.2)

ax.set_xscale('log')
ax.set_ylim (0.3,1.01)
ax.set_xlim(1,2000)
ax.legend(loc='best')
ax.tick_params(axis = "both", which = "both", direction = "in")
plt.xlabel("Distance [pc]", fontsize = 14)
ax.set_ylabel(r"CDF f $(< d)$", fontsize = 14)
ax.grid(which='major', alpha=0.5)
plt.tight_layout()
fig.savefig("../Figures/ModelDistanceCDF.png", dpi=300)
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))

sns.ecdfplot(realSNeInt, color = "dimgrey",  label="Real SNe Sample", ax=ax,lw=2)
sns.ecdfplot(im2, label="H-alpha", ls="--",lw=2,ax=ax)
sns.ecdfplot(im3, label="Stars (3.4 Micron)", ls="-.",lw=2,ax=ax)
sns.ecdfplot(im1, label="Random", ls=":",ax=ax,lw=2)
ax.fill_betweenx(y=iSMm2_CDF, x1=iSMm2_16,x2=iSMm2_84, alpha=0.2)
ax.fill_betweenx(y=iSMm3_CDF, x1=iSMm3_16,x2=iSMm3_84, alpha=0.2)
ax.fill_betweenx(y=iSMm1_CDF, x1=iSMm1_16,x2=iSMm1_84, alpha=0.2)
ax.set_xscale('log')
ax.set_ylim (0.,1.01)
ax.set_xlim(1,300000)
ax.legend(loc='best')
ax.tick_params(axis = "both", which = "both", direction = "in")
plt.xlabel(r'$I_{H_a}$ $[10^{-20} erg s^{-1} cm^{-2} spaxel^{-1}]$', fontsize = 14)
ax.set_ylabel(r'CDF f($<I_{H_a}$)', fontsize = 14)
ax.grid(which='major', alpha=0.5)
plt.tight_layout()
fig.savefig("../Figures/ModelIntensityCDF.png", dpi=300)
plt.show()
plt.close()